In [ ]:
!pip install pydicom opencv-python-headless[app] nibabel matplotlib albumentations tqdm

In [10]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
import glob
import cv2
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from albumentations import HorizontalFlip, VerticalFlip, Rotate
from tqdm import tqdm
import torch.nn.functional as F
import matplotlib.image as mpimg
import albumentations as A
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import jaccard_score
from tqdm import tqdm
import nibabel as nib
import albumentations as A
from PIL import Image
from torchvision.transforms import ToTensor, Normalize

height,width = (256,256) 


torch.cuda.is_available()

True

In [12]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)
    
    
class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.double_conv = DoubleConv(in_channels, out_channels)
        self.down_sample = nn.MaxPool2d(2)

    def forward(self, x):
        skip_out = self.double_conv(x)
        down_out = self.down_sample(skip_out)
        return (down_out, skip_out)

    
class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels, up_sample_mode):
        super(UpBlock, self).__init__()
        if up_sample_mode == 'conv_transpose':
            self.up_sample = nn.ConvTranspose2d(in_channels-out_channels, in_channels-out_channels, kernel_size=2, stride=2)        
        elif up_sample_mode == 'bilinear':
            self.up_sample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            raise ValueError("Unsupported `up_sample_mode` (can take one of `conv_transpose` or `bilinear`)")
        self.double_conv = DoubleConv(in_channels, out_channels)

    def forward(self, down_input, skip_input):
        x = self.up_sample(down_input)
        x = torch.cat([x, skip_input], dim=1)
        return self.double_conv(x)

    
class UNet(nn.Module):
    def __init__(self, out_classes=3, up_sample_mode='conv_transpose'):
        super(UNet, self).__init__()
        self.up_sample_mode = up_sample_mode
        # Downsampling Path
        self.down_conv1 = DownBlock(3, 64)
        self.down_conv2 = DownBlock(64, 128)
        self.down_conv3 = DownBlock(128, 256)
        self.down_conv4 = DownBlock(256, 512)
        # Bottleneck
        self.double_conv = DoubleConv(512, 1024)
        # Upsampling Path
        self.up_conv4 = UpBlock(512 + 1024, 512, self.up_sample_mode)
        self.up_conv3 = UpBlock(256 + 512, 256, self.up_sample_mode)
        self.up_conv2 = UpBlock(128 + 256, 128, self.up_sample_mode)
        self.up_conv1 = UpBlock(128 + 64, 64, self.up_sample_mode)
        # Final Convolution
        self.conv_last = nn.Conv2d(64, out_classes, kernel_size=1)
        
        

    def forward(self, x):
        x, skip1_out = self.down_conv1(x)
        x, skip2_out = self.down_conv2(x)
        x, skip3_out = self.down_conv3(x)
        x, skip4_out = self.down_conv4(x)
        x = self.double_conv(x)
        x = self.up_conv4(x, skip4_out)
        x = self.up_conv3(x, skip3_out)
        x = self.up_conv2(x, skip2_out)
        x = self.up_conv1(x, skip1_out)
        x = self.conv_last(x)
        
        return x
    

# Get UNet model
model = UNet()

In [13]:
import pydicom  # pydicom kütüphanesi DICOM formatındaki tıbbi görüntüleri okumak için kullanılır.
import nibabel as nib  # nibabel kütüphanesi NIfTI formatındaki tıbbi görüntüleri okumak için kullanılır.
import numpy as np  # numpy kütüphanesi matematiksel işlemler ve veri işleme için kullanılır.
import matplotlib.pyplot as plt  # matplotlib kütüphanesi verileri görselleştirmek için kullanılır.
from skimage import measure  # skimage kütüphanesi görüntü işleme ve analizi için kullanılır.

class Temp:
    _temp_img=None  # Sınıf içinde kullanılacak geçici değişkenler için Temp sınıfı oluşturuldu.
    _temp_mock=None
    rel_vals=None

def get_largest_mask(mask):
    """
    Fonksiyon, en büyük maskeyi elde etmek için girdi olarak aldığı maske görüntüsünün etiketlerini ve 
    özelliklerini hesaplar ve sonra maske üzerindeki bölgeleri büyüklüklerine göre sıralar. 
    Son olarak, en büyük bölgeyi elde etmek için diğer tüm bölgeleri sıfırlar.
    """
    labels_mask = measure.label(mask)  # maske görüntüsünün etiketlerini hesaplar
    regions = measure.regionprops(labels_mask)  # etiketli bölgelerin özelliklerini hesaplar
    regions.sort(key=lambda x: x.area, reverse=True)  # bölgeleri alanlarına göre sıralar
    if len(regions) > 1:  # en büyük bölgenin dışında diğer bölgeleri sıfırlar
        for rg in regions[1:]:
            labels_mask[rg.coords[:,0], rg.coords[:,1]] = 0
    labels_mask[labels_mask!=0] = 1  # maske görüntüsünde kalan tüm bölgeleri birleştirir
    mask = labels_mask
    return mask 

def crop_and_mask_dcm_image(masks, real_img):
    """
    Fonksiyon, DICOM görüntüsünü ve onun maske görüntüsünü alır. Görüntü boyutları eşit olmadığı durumlarda
    maske boyutunu gerçek görüntüye uyarlar. Daha sonra en büyük maskeyi elde etmek için 'get_largest_mask' 
    fonksiyonunu kullanır. Son olarak, görüntüyü maskeyle çarparak maske dışındaki alanları siyah renge döker.
    """
    real_w,real_h=real_img.shape  # gerçek görüntünün boyutlarını hesaplar
    mock_masks=cv2.resize(masks,(real_h,real_w), interpolation=cv2.INTER_AREA)  # maske boyutunu gerçek görüntüye uyarlama işlemi yapar
    
    Temp._temp_mock=mock_masks
    mock_masks=get_largest_mask(mock_masks)  # en büyük maskeyi elde eder
    rslt = mock_masks* real_img  # maske görüntüsü ve gerçek görüntüyü çarparak sonucu elde eder

    rows, cols = np.where(mock_masks!= 0)  # maske görüntüsünde sıfırdan farklı değerlerin satır ve sütun indislerini elde eder
    y1, x1 = np.min(rows), np.min(cols)  # satır ve sütun indislerinin en küçük değerlerini hesaplar
    y2, x2 = np.max(rows), np.max(cols)  # satır ve sütun indislerinin en büyük değerlerini hesaplar

    # Görüntüyü bounding box ile kırp
    result = rslt[y1:y2+1, x1:x2+1]  # bounding box işlemi yaparak sonucu elde eder

    # Maske dışındaki alanı siyah renge dök
    result[mock_masks[y1:y2+1, x1:x2+1] == 0] = 0  # maske dışındaki alanları siyah renge dökme işlemi yapar

    Temp._temp_img=result

    return rslt, result  # sonuçları döndürür


In [14]:
PATH="/home/uraninjo/Documents/VEri"# TODO: Change this
#PATH="/media/uraninjo/7D72-19E7/teknofest"
test_images_path=sorted(glob.glob(f'{PATH}/*/*.dcm'))  # Görüntü yollarının listesi

classes = [0, 1, 2]  # Orijinal etiket değerleri
print(len(test_images_path))

1600


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
model = UNet()
model.load_state_dict(torch.load("/home/uraninjo/Desktop/Çalışmalar/Mergen1-Teknofest/Murat/checkpoints3/Kullanılacaklar/Segmentasyon_Zeynep2Kanal_6_ValLoss0.5481_diceScore0.9260.pt"))
model.to(device)
model.eval()
print(" ")

In [17]:
transform = A.Compose([
            A.Resize(height=256,width=256,p=1.0),
            A.augmentations.transforms.CLAHE(clip_limit=(2.0,3.0), tile_grid_size=(8, 8), always_apply=False, p=0.5) 
        ])
to_tensor = ToTensor()

In [ ]:
import time
t1=time.time()

# SegmentationOutput klasörünün oluşturulması
try:
    os.mkdir(f"/home/uraninjo/Desktop/Çalışmalar/Mergen1-Teknofest/Murat/TestPipeline/SegmentasyonOutput")#{__hno}
    # TODO: Change this
except:
    pass

with torch.no_grad():
    # Tüm test görüntülerinin üzerinde döngüye girilmesi
    for i in range(len(test_images_path)):
        try:
            # Görüntü türü ve hasta numarasının elde edilmesi
            __tür=test_images_path[i].split("/")[-1].split(".")[0]
            __hno=test_images_path[i].split("/")[-2]
            
            # Görüntünün okunması ve piksel dizisine dönüştürülmesi
            img = pydicom.dcmread(test_images_path[i], force=True)
            monochrome=img.PhotometricInterpretation
            img=img.pixel_array
            if monochrome=='MONOCHROME1':
                img=np.invert(img)
            img = ((img - img.min()) * (1/(img.max() - img.min()) * 255)).astype('uint8')
            
            # Görüntünün boyutunun yeniden boyutlandırılması ve tensor formatına dönüştürülmesi
            transformed = transform(image=img)
            img_t = transformed["image"]
            img_t = np.array([img_t,img_t,img_t])
            img_t=np.moveaxis(img_t,0,-1)
            img_t = to_tensor(img_t)
            img_t = torch.unsqueeze(img_t, 0)
            inputs = img_t.to(device)
            
            # Görüntü segmentasyonunun yapılması
            outputs = model(inputs)
            output_np = outputs.detach().cpu().numpy()
            output_np = output_np - np.min(output_np)
            output_np = output_np / np.max(output_np)
            predicted_masks = np.around(output_np, decimals=0, out=None)
            mask_1 = predicted_masks[0][1,:,:]
            mask_2 = predicted_masks[0][2,:,:]
            combined_mask = np.maximum(mask_1, mask_2)

            # Görüntü kırpma ve maskeleme işlemlerinin yapılması
            rslt, result = crop_and_mask_dcm_image(combined_mask,img)#inputs[0][0,:,:].cpu().numpy()
            result_resize = cv2.resize(result.astype('uint8'),(250,500))
            
            # Hasta klasörünün oluşturulması
            try:
                os.mkdir(f"/home/uraninjo/Desktop/Çalışmalar/Mergen1-Teknofest/Murat/TestPipeline/SegmentasyonOutput/{__hno}")#{__hno}
                # TODO: Change this
            except:
                pass
                # print("FoldAttributeError: 'FileDataset' object has no attribute 'PhotometricInterpretation'er already exists")
                
            # Sonuç görüntüsünün kaydedilmesi
            plt.imsave(f"/home/uraninjo/Desktop/Çalışmalar/Mergen1-Teknofest/Murat/TestPipeline/SegmentasyonOutput/{__hno}/{__tür}.png",result_resize,cmap="gray")
                        # Ekrana bilgi yazdırılması
            print(f"Dosya oluşturuldu sıra:{i}-{__tür}-{__hno}")

        except Exception as e:
            print(e)
            print("Hatalı", test_images_path[i])
            continue
            
t2=time.time()
print(t2-t1)
